# Encontramos tu oficina

**Carga de datos...**

**Pasando a PostGres...**

In [1]:
#query('SELECT PostGIS_Full_Version();')
#query(f'CREATE EXTENSION postgis WITH SCHEMA {SCHEMA};')
#query('ALTER DATABASE officesbi SET search_path TO public, postgis;')

In [2]:
# Enable PostGIS (as of 3.0 contains just geometry/geography)
#motor.execute(DDL("CREATE EXTENSION postgis;"))
# enable raster support (for 3+)
#motor.execute(DDL("CREATE EXTENSION postgis_raster;"))
# Enable Topology
#motor.execute(DDL("CREATE EXTENSION postgis_topology;"))
# Enable PostGIS Advanced 3D and other geoprocessing algorithms sfcgal not available with all distributions
#motor.execute(DDL("CREATE EXTENSION postgis_sfcgal;"))
# fuzzy matching needed for Tiger
#motor.execute(DDL("CREATE EXTENSION fuzzystrmatch;"))
# rule based standardizer
#motor.execute(DDL("CREATE EXTENSION address_standardizer;"))
# example rule data set
#motor.execute(DDL("CREATE EXTENSION address_standardizer_data_us;"))
# Enable US Tiger Geocoder
#motor.execute(DDL("CREATE EXTENSION postgis_tiger_geocoder;"))